In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

df = pd.read_csv('/kaggle/input/capstone-bank-lending-prediction/lending_train.csv')

In [28]:
import seaborn as sns

In [44]:
from scipy.stats import chi2_contingency
from scipy.stats import chisquare

In [31]:
from scipy.stats import f_oneway

In [33]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

numeric_cols = df.select_dtypes(include=numerics).columns # get column names containing numeric data

In [36]:
for col in numeric_cols:
    CategoryGroupLists=df.groupby('race')[col].apply(list)
    # perform the ANOVA test
    AnovaResults = f_oneway(*CategoryGroupLists)
    print(col, ' P-Value for Anova is: ', AnovaResults[1])

ID  P-Value for Anova is:  0.2630202471325162
requested_amnt  P-Value for Anova is:  0.1256348228567462
annual_income  P-Value for Anova is:  0.16895793560701916
debt_to_income_ratio  P-Value for Anova is:  nan
public_bankruptcies  P-Value for Anova is:  nan
delinquency_last_2yrs  P-Value for Anova is:  0.790908284785353
fico_score_range_low  P-Value for Anova is:  0.0
fico_score_range_high  P-Value for Anova is:  0.0
fico_inquired_last_6mths  P-Value for Anova is:  nan
months_since_last_delinq  P-Value for Anova is:  nan
revolving_balance  P-Value for Anova is:  0.1632079978389206
total_revolving_limit  P-Value for Anova is:  nan
any_tax_liens  P-Value for Anova is:  nan
loan_paid  P-Value for Anova is:  0.33887898390201676


fico_score_range_low and fico_score_range_high correlated w race column

In [45]:
categorical_cols = list(set(df.columns) - set(numeric_cols))

for col in categorical_cols:
    crosstab_result = pd.crosstab(index=df['race'], columns=df[col])
    # Performing Chi-sq test
    ChiSqRes = chi2_contingency(crosstab_result)
    print(col, ' p-value: ', ChiSqRes[1]) # p-value

employment_verified  p-value:  0.6837111491424117
zipcode  p-value:  0.4697984609521372
type_of_application  p-value:  0.5988728505564539
race  p-value:  0.0
state  p-value:  0.010921425125704647
employment_length  p-value:  0.3225419228800779
loan_duration  p-value:  0.5432004833448337
home_ownership_status  p-value:  0.16806783081281643
extended_reason  p-value:  0.028220893660891835
employment  p-value:  0.19558327068706524
reason_for_loan  p-value:  0.5791426027145052


state and extended_reason are correlated w race column

In [41]:
df[numeric_cols].corr()

,ID,requested_amnt,annual_income,debt_to_income_ratio,public_bankruptcies,delinquency_last_2yrs,fico_score_range_low,fico_score_range_high,fico_inquired_last_6mths,months_since_last_delinq,revolving_balance,total_revolving_limit,any_tax_liens,loan_paid
ID,1.000000,-0.000911,-0.000064,-0.001160,0.002166,0.001178,-0.000262,-0.000262,0.002824,-0.001355,0.000400,0.001100,0.000519,0.000389
requested_amnt,-0.000911,1.000000,0.309582,0.031978,-0.094622,-0.003870,0.083669,0.083669,-0.014549,-0.022934,0.315641,0.306145,0.007517,-0.065879
annual_income,-0.000064,0.309582,1.000000,-0.139150,-0.043623,0.040529,0.059814,0.059814,0.032989,-0.041328,0.283317,0.266772,0.038965,0.041332
debt_to_income_ratio,-0.001160,0.031978,-0.139150,1.000000,-0.018617,-0.007724,-0.053618,-0.053621,-0.006618,0.007928,0.109730,0.053821,-0.023499,-0.084216
public_bankruptcies,0.002166,-0.094622,-0.043623,-0.018617,1.000000,-0.047505,-0.169345,-0.169343,0.073753,0.121351,-0.116304,-0.115541,0.035285,-0.025268
delinquency_last_2yrs,0.001178,-0.003870,0.040529,-0.007724,-0.047505,1.000000,-0.145713,-0.145711,0.024242,-0.555125,-0.027729,-0.042326,0.011834,-0.019654
fico_score_range_low,-0.000262,0.083669,0.059814,-0.053618,-0.169345,-0.145713,1.000000,1.000000,-0.071981,0.074179,0.020236,0.229080,-0.049894,0.110292
fico_score_range_high,-0.000262,0.083669,0.059814,-0.053621,-0.169343,-0.145711,1.000000,1.000000,-0.071982,0.074179,0.020234,0.229082,-0.049894,0.110291
fico_inquired_last_6mths,0.002824,-0.014549,0.032989,-0.006618,0.073753,0.024242,-0.071981,-0.071982,1.000000,0.016381,-0.010593,0.017089,0.010190,-0.064352
months_since_last_delinq,-0.001355,-0.022934,-0.041328,0.007928,0.121351,-0.555125,0.074179,0.074179,0.016381,1.000000,-0.019499,-0.017876,0.006061,0.011882


revolving_balance and total_revolving_limit very highly correlated, total_revolving limit more correlated with loan_paid, so we keep this one

same logic with months_since_last_delinq and delinquency_last_2yrs, keep the latter

In [48]:
cat_df = df[categorical_cols]  # temp df of only categorical data

cat_df.apply(lambda x : pd.factorize(x)[0]).corr(method='pearson', min_periods=1)


,employment_verified,zipcode,type_of_application,race,state,employment_length,loan_duration,home_ownership_status,extended_reason,employment,reason_for_loan
employment_verified,1.000000,0.002744,-0.010139,-0.000514,0.000675,0.056280,0.128295,-0.000796,-0.009509,0.012167,0.025973
zipcode,0.002744,1.000000,0.016068,-0.000114,0.241130,-0.029487,-0.012543,0.098551,-0.008442,-0.007293,-0.009565
type_of_application,-0.010139,0.016068,1.000000,0.000424,0.014198,-0.005190,-0.042277,0.032773,-0.031041,-0.025746,-0.010344
race,-0.000514,-0.000114,0.000424,1.000000,0.000624,0.000852,-0.000076,0.000052,-0.000318,-0.000027,-0.000787
state,0.000675,0.241130,0.014198,0.000624,1.000000,-0.004495,-0.009861,0.052800,-0.007033,-0.000689,-0.012488
employment_length,0.056280,-0.029487,-0.005190,0.000852,-0.004495,1.000000,0.008366,-0.095858,0.021631,0.067546,0.018040
loan_duration,0.128295,-0.012543,-0.042277,-0.000076,-0.009861,0.008366,1.000000,-0.057055,0.012546,-0.008997,0.071129
home_ownership_status,-0.000796,0.098551,0.032773,0.000052,0.052800,-0.095858,-0.057055,1.000000,-0.019270,-0.024415,-0.026819
extended_reason,-0.009509,-0.008442,-0.031041,-0.000318,-0.007033,0.021631,0.012546,-0.019270,1.000000,0.195486,0.062599
employment,0.012167,-0.007293,-0.025746,-0.000027,-0.000689,0.067546,-0.008997,-0.024415,0.195486,1.000000,0.005092


In [39]:
excluded = ['months_since_last_delinq', 'revolving_balance', 'race', 'state', 'extended_reason', 'fico_score_range_low', 'fico_score_range_high']
included = list(set(df.columns) - set(excluded))

df = df[included]

print(df.head(5))

                       employment  annual_income employment_length  \
0                             SVP       250000.0           4 years   
1    METAL FABRICATION SUPERVISOR        70000.0         10+ years   
2  insurance collector specialist        37000.0         10+ years   
3                 Kitchen Manager        45000.0         10+ years   
4              Appeals Specialist        86000.0         10+ years   

   any_tax_liens  total_revolving_limit home_ownership_status  ID  \
0            0.0               105700.0                  RENT   0   
1            0.0                14600.0              MORTGAGE   1   
2            0.0                10300.0                   OWN   2   
3            0.0                25200.0                   OWN   3   
4            0.0               104200.0              MORTGAGE   4   

      reason_for_loan  revolving_balance  delinquency_last_2yrs  \
0  debt_consolidation            22480.0                    0.0   
1    home_improvement         